In [1]:
import os
import rootpath
import sys
sys.path.append(rootpath.detect())

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from testsuite.results import Result, ResultsContainer
from testsuite.analysis_tools import strip_problem_names
from testsuite.utilities import Pareto_split

In [5]:
from figure_utilities import *

In [6]:
from table_utilities import save_table, get_comparison_matrices

In [7]:
def get_med_iqr(file_name):
    TABLE_STAGES = [25, 50, 100, 150]
    result = load_igd_result(file_name)
    stage_inds = [np.where(np.asarray(result.igd_hist_x)==ts)[0][0] for ts in TABLE_STAGES] 
    
    med = np.median(result.igd_history[:, stage_inds], axis=0)
    lqr = np.quantile(result.igd_history[:, stage_inds], 0.25, axis=0)
    uqr = np.quantile(result.igd_history[:, stage_inds], 0.75, axis=0)
    
    med_r = np.median(result.igdref_history[:, stage_inds], axis=0)
    lqr_r = np.quantile(result.igdref_history[:, stage_inds], 0.25, axis=0)
    uqr_r = np.quantile(result.igdref_history[:, stage_inds], 0.75, axis=0)
    
    return med, uqr-lqr, med_r, uqr_r-lqr_r

def get_table_from_list(LIST):
    D = {}
    for file_name in LIST:
        key = file_name[:-7]
        med, iqr, med_r, iqr_r = get_med_iqr(file_name)
        data0 = np.hstack((med.reshape(-1,1), iqr.reshape(-1,1)))
        data1 = np.hstack((med_r.reshape(-1,1), iqr_r.reshape(-1,1)))
        if key+"_1" not in D.keys(): 
            D[key+f"_1"] = np.hstack([di for di in data0]+[di for di in data1])
        else:
            D[key+f"_2"] = np.hstack([di for di in data0]+[di for di in data1])
    return D

In [8]:
def apply_f_to_mask(df, mask, f):
    best_inds =  [*zip(*np.where(mask == 1))] 
    for i, j in best_inds:
        df.iloc[i, j] = f(df.iloc[i, j])

In [9]:
def format_table(df):
    df_str = df.astype(str)
    # italicise IQR columns
    mask_italics = np.zeros_like(df_str.values, dtype=bool)
    mask_italics[:,1::2] = True
    f_italics = lambda cell_value: r"\textit{"+cell_value+"}"
    apply_f_to_mask(df_str, mask_italics, f_italics)
    
    # get boolean mask of winners/same results for each
    values_directed = df_str.values[:,:int(df_str.values.shape[1]/2)]
    values_undirected = df_str.values[:,int(df_str.values.shape[1]/2):]
    
    # highlight best
    directed_wins = np.repeat((values_directed<values_undirected)[:, ::2], 2, axis=1)
    undirected_wins = np.repeat((values_directed>values_undirected)[:, ::2], 2, axis=1)
    mask_best = np.hstack([directed_wins, undirected_wins])
    f_best = lambda cell_value: r"\best "+cell_value
    apply_f_to_mask(df_str, mask_best, f_best)
    
    # highlight draws
    draw = np.repeat((values_directed==values_undirected)[:, ::2], 2, axis=1)
    mask_draw = np.hstack([draw, draw])
    f_draw = lambda cell_value: r"\statsimilar "+cell_value
    apply_f_to_mask(df_str, mask_draw, f_draw)

    return df_str

In [10]:
def load_igd_result(file_name):
    assert file_name in RESULT_FILES_LIST
    path = os.path.join(PATH_TO_IGD_RESULTS, file_name)
    assert os.path.isfile(path)
    return ResultsContainer(path)

In [11]:
D = get_table_from_list(UNATTAINABLE_RESULTS_FILE_LIST)
mic = pd.MultiIndex.from_product([["Directed", "Undirected"], [25, 50, 100, 150],['median', 'IQR']])
mir = pd.MultiIndex.from_product([[d[:-2].replace("_", " ") for d in D.keys()][::2], ["1", "2"]], names=[r"$f$", "Position"])
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_unattainable = format_table(df2)
# save_table(df_unattainable, "unattainable_table_igd")
df_unattainable

Directed                        \
                                  25                          
                               median                   IQR   
$f$             Position                                      
wfg1 2obj 3dim  1               1.246        \textit{0.109}   
                2               1.045        \textit{0.094}   
wfg1 3obj 4dim  1         \best 1.345   \best \textit{0.29}   
                2               1.527         \textit{0.11}   
wfg1 4obj 5dim  1         \best 1.894  \best \textit{0.247}   
                2               2.027        \textit{0.149}   
wfg2 2obj 6dim  1          \best 0.52  \best \textit{0.159}   
                2         \best 0.808  \best \textit{0.341}   
wfg2 3obj 6dim  1                1.46        \textit{0.621}   
                2         \best 0.861  \best \textit{0.245}   
wfg2 4obj 10dim 1          \best 2.44  \best \textit{0.674}   
                2         \best 0.552  \best \textit{0.207}   
wfg3 2obj 6dim  1         \best 0.233  \best \textit{0.137}   
                2         \best 0.204  \best \textit{0.091}   
wfg3 3obj 10dim 1         \best 0.412  \best \textit{0.166}   
                2         \best 0.527  \best \textit{0.343}   
wfg3 4obj 10dim 1         \best 0.496  \best \textit{0.241}   
                2         \best 0.726  \best \textit{0.514}   
wfg4 2obj 6dim  1         \best 0.511  \best \textit{0.313}   
                2         \best 0.222  \best \textit{0.082}   
wfg4 3obj 8dim  1         \best 0.575  \best \textit{0.371}   
                2         \best 0.994  \best \textit{0.801}   
wfg4 4obj 8dim  1         \best 1.245  \best \textit{0.696}   
                2          \best 1.26   \best \textit{0.57}   
wfg5 2obj 6dim  1          \best 0.31    \best \textit{0.2}   
                2         \best 0.178  \best \textit{0.195}   
wfg5 3obj 8dim  1         \best 0.766  \best \textit{0.317}   
                2          \best 0.69  \best \textit{0.379}   
wfg5 4obj 10dim 1         \best 1.598  \best \textit{0.713}   
                2         \best 1.152  \best \textit{0.493}   
wfg6 2obj 6dim  1         \best 0.347  \best \textit{0.131}   
                2         \best 0.433  \best \textit{0.276}   
wfg6 3obj 8dim  1         \best 0.844  \best \textit{0.275}   
                2         \best 0.522  \best \textit{0.292}   
wfg6 4obj 10dim 1         \best 1.504  \best \textit{0.606}   
                2         \best 1.157  \best \textit{0.495}   

                                                                          \
                                         50                                
                                      median                         IQR   
$f$             Position                                                   
wfg1 2obj 3dim  1                       1.23              \textit{0.122}   
                2                      1.016              \textit{0.086}   
wfg1 3obj 4dim  1                \best 1.285        \best \textit{0.237}   
                2                      1.488              \textit{0.138}   
wfg1 4obj 5dim  1                \best 1.823        \best \textit{0.147}   
                2                \best 1.944         \best \textit{0.11}   
wfg2 2obj 6dim  1                \best 0.411        \best \textit{0.234}   
                2                \best 0.701        \best \textit{0.327}   
wfg2 3obj 6dim  1                      1.408              \textit{0.748}   
                2                \best 0.831        \best \textit{0.282}   
wfg2 4obj 10dim 1         \statsimilar 2.318  \statsimilar \textit{0.88}   
                2                \best 0.403        \best \textit{0.181}   
wfg3 2obj 6dim  1                 \best 0.09        \best \textit{0.038}   
                2                  \best 0.1        \best \textit{0.054}   
wfg3 3obj 10dim 1                \best 0.222        \best \textit{0.088}   
                2                \best 0.235        \best \texti

In [12]:
D = get_table_from_list(PARETO_RESULTS_FILE_LIST)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_pareto = format_table(df2)
# save_table(df_pareto, "pareto_table_igd")
df_pareto

Directed                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.297        \textit{0.135}        1.264   
                2                1.05        \textit{0.076}        1.016   
wfg1 3obj 4dim  1         \best 1.372  \best \textit{0.095}  \best 1.341   
                2               1.531        \textit{0.182}        1.485   
wfg1 4obj 5dim  1         \best 1.906  \best \textit{0.247}  \best 1.822   
                2               2.019        \textit{0.126}   \best 1.94   
wfg2 2obj 6dim  1         \best 0.496  \best \textit{0.294}  \best 0.359   
                2         \best 0.892  \best \textit{0.374}  \best 0.807   
wfg2 3obj 6dim  1               1.529        \textit{0.558}  \best 1.358   
                2         \best 0.841  \best \textit{0.372}  \best 0.789   
wfg2 4obj 10dim 1               2.597        \textit{0.575}        2.411   
                2         \best 0.457   \best \textit{0.28}  \best 0.235   
wfg3 2obj 6dim  1         \best 0.214  \best \textit{0.139}  \best 0.068   
                2         \best 0.208   \best \textit{0.11}  \best 0.059   
wfg3 3obj 10dim 1         \best 0.484  \best \textit{0.171}  \best 0.187   
                2         \best 0.531  \best \textit{0.263}  \best 0.242   
wfg3 4obj 10dim 1         \best 0.496   \best \textit{0.31}  \best 0.171   
                2         \best 0.946  \best \textit{0.378}  \best 0.383   
wfg4 2obj 6dim  1         \best 0.543  \best \textit{0.347}  \best 0.374   
                2         \best 0.209  \best \textit{0.073}  \best 0.153   
wfg4 3obj 8dim  1         \best 0.759  \best \textit{0.883}   \best 0.44   
                2         \best 0.903   \best \textit{0.82}  \best 0.755   
wfg4 4obj 8dim  1          \best 2.17  \best \textit{1.956}  \best 0.722   
                2         \best 1.282  \best \textit{0.465}  \best 0.743   
wfg5 2obj 6dim  1         \best 0.307  \best \textit{0.219}  \best 0.113   
                2         \best 0.209  \best \textit{0.251}  \best 0.078   
wfg5 3obj 8dim  1         \best 0.974  \best \textit{0.462}  \best 0.277   
                2         \best 0.632   \best \textit{0.29}  \best 0.229   
wfg5 4obj 10dim 1          \best 2.69  \best \textit{1.115}  \best 1.466   
                2         \best 1.422  \best \textit{0.471}  \best 0.748   
wfg6 2obj 6dim  1         \best 0.303  \best \textit{0.246}  \best 0.124   
                2         \best 0.539  \best \textit{0.328}  \best 0.224   
wfg6 3obj 8dim  1          \best 0.99  \best \textit{0.519}   \best 0.32   
                2         \best 0.588  \best \textit{0.308}  \best 0.256   
wfg6 4obj 10dim 1         \best 2.036  \best \textit{0.957}   \best 0.81   
                2         \best 1.558  \best \textit{0.358}  \best 0.775   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1               \textit{0.156}        1.255   
                2                \textit{0.06}  \best 0.983   
wfg1 3obj 4dim  1          \best \textit{0.08}  \best 1.332   
                2               \textit{0.198}        1.466   
wfg1 4obj 5dim  1         \best \textit{0.183}  \best 1.782   
                2          \best \textit{0.18}  \best 1.919   
wfg2 2obj 6dim  1         \best \textit{0.245}   \best 0.16   
                2         \best \textit{0.365}  \best 0.711   
wfg2 3obj 6dim  1         \best \textit{0.651}  \best 1.212   
                2         \best \textit{0.284}  \best 0.647   
wfg2 4obj 10dim 1               \textit{0.706}   \best 2.01   
                2         \best \textit{0.1

In [13]:
D = get_table_from_list(ATTAINABLE_RESULTS_FILE_LIST)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_attainable_a = format_table(df2)
# save_table(df_attainable_a, "attainable_table_volA_igd")
df_attainable_a

Directed                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.301        \textit{0.147}  \best 1.262   
                2               1.027        \textit{0.089}        1.021   
wfg1 3obj 4dim  1         \best 1.404  \best \textit{0.358}        1.379   
                2               1.494        \textit{0.137}  \best 1.444   
wfg1 4obj 5dim  1          \best 1.84  \best \textit{0.181}  \best 1.793   
                2         \best 1.937  \best \textit{0.171}        1.893   
wfg2 2obj 6dim  1         \best 0.527  \best \textit{0.259}  \best 0.317   
                2          \best 1.05  \best \textit{0.534}  \best 1.028   
wfg2 3obj 6dim  1               1.714        \textit{0.876}        1.667   
                2                1.04        \textit{0.442}         1.01   
wfg2 4obj 10dim 1         \best 2.393  \best \textit{0.509}        2.351   
                2         \best 0.425  \best \textit{0.225}  \best 0.223   
wfg3 2obj 6dim  1         \best 0.206  \best \textit{0.096}   \best 0.08   
                2         \best 0.244  \best \textit{0.135}  \best 0.103   
wfg3 3obj 10dim 1         \best 0.478  \best \textit{0.295}  \best 0.193   
                2         \best 0.621  \best \textit{0.333}  \best 0.228   
wfg3 4obj 10dim 1         \best 0.506  \best \textit{0.239}  \best 0.176   
                2           \best 0.9  \best \textit{0.504}  \best 0.348   
wfg4 2obj 6dim  1         \best 0.776  \best \textit{0.698}        0.503   
                2         \best 0.184  \best \textit{0.057}  \best 0.142   
wfg4 3obj 8dim  1         \best 1.317  \best \textit{1.233}  \best 0.834   
                2         \best 1.079  \best \textit{1.445}  \best 0.753   
wfg4 4obj 8dim  1         \best 2.216  \best \textit{0.942}  \best 0.943   
                2         \best 1.569  \best \textit{1.158}  \best 0.733   
wfg5 2obj 6dim  1         \best 0.464  \best \textit{0.431}  \best 0.186   
                2           \best 0.2   \best \textit{0.19}  \best 0.092   
wfg5 3obj 8dim  1         \best 1.274  \best \textit{0.699}  \best 0.428   
                2         \best 1.166  \best \textit{0.405}  \best 0.442   
wfg5 4obj 10dim 1         \best 2.624  \best \textit{0.839}  \best 1.557   
                2         \best 1.737   \best \textit{0.47}  \best 0.755   
wfg6 2obj 6dim  1         \best 0.439  \best \textit{0.172}  \best 0.191   
                2         \best 0.496   \best \textit{0.24}  \best 0.298   
wfg6 3obj 8dim  1         \best 1.172  \best \textit{0.723}  \best 0.403   
                2         \best 0.786   \best \textit{0.38}  \best 0.406   
wfg6 4obj 10dim 1         \best 2.333  \best \textit{0.928}  \best 0.661   
                2         \best 1.677   \best \textit{0.62}  \best 0.829   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1         \best \textit{0.199}        1.262   
                2                \textit{0.07}        0.981   
wfg1 3obj 4dim  1               \textit{0.253}        1.369   
                2         \best \textit{0.157}        1.444   
wfg1 4obj 5dim  1         \best \textit{0.165}  \best 1.776   
                2               \textit{0.129}  \best 1.831   
wfg2 2obj 6dim  1         \best \textit{0.305}  \best 0.148   
                2         \best \textit{0.508}  \best 0.893   
wfg2 3obj 6dim  1                \textit{0.87}  \best 1.338   
                2                \textit{0.36}  \best 0.833   
wfg2 4obj 10dim 1               \textit{0.632}         2.14   
                2         \best \textit{0.1

In [16]:
D = get_table_from_list(ATTAINABLE_RESULTS_FILE_LIST)
df = pd.DataFrame.from_dict(D, orient="index", columns=mic).round(3)
df2 = pd.DataFrame(df.values, index=mir, columns=mic).round(3)
df2

df_attainable_b = format_table(df2)
# save_table(df_attainable_b, "attainable_table_volB_igd")
df_attainable_b

Directed                                     \
                                  25                                 50    
                               median                   IQR       median   
$f$             Position                                                   
wfg1 2obj 3dim  1               1.301        \textit{0.147}  \best 1.262   
                2               1.027        \textit{0.089}        1.021   
wfg1 3obj 4dim  1         \best 1.404  \best \textit{0.358}        1.379   
                2               1.494        \textit{0.137}  \best 1.444   
wfg1 4obj 5dim  1          \best 1.84  \best \textit{0.181}  \best 1.793   
                2         \best 1.937  \best \textit{0.171}        1.893   
wfg2 2obj 6dim  1         \best 0.527  \best \textit{0.259}  \best 0.317   
                2          \best 1.05  \best \textit{0.534}  \best 1.028   
wfg2 3obj 6dim  1               1.714        \textit{0.876}        1.667   
                2                1.04        \textit{0.442}         1.01   
wfg2 4obj 10dim 1         \best 2.393  \best \textit{0.509}        2.351   
                2         \best 0.425  \best \textit{0.225}  \best 0.223   
wfg3 2obj 6dim  1         \best 0.206  \best \textit{0.096}   \best 0.08   
                2         \best 0.244  \best \textit{0.135}  \best 0.103   
wfg3 3obj 10dim 1         \best 0.478  \best \textit{0.295}  \best 0.193   
                2         \best 0.621  \best \textit{0.333}  \best 0.228   
wfg3 4obj 10dim 1         \best 0.506  \best \textit{0.239}  \best 0.176   
                2           \best 0.9  \best \textit{0.504}  \best 0.348   
wfg4 2obj 6dim  1         \best 0.776  \best \textit{0.698}        0.503   
                2         \best 0.184  \best \textit{0.057}  \best 0.142   
wfg4 3obj 8dim  1         \best 1.317  \best \textit{1.233}  \best 0.834   
                2         \best 1.079  \best \textit{1.445}  \best 0.753   
wfg4 4obj 8dim  1         \best 2.216  \best \textit{0.942}  \best 0.943   
                2         \best 1.569  \best \textit{1.158}  \best 0.733   
wfg5 2obj 6dim  1         \best 0.464  \best \textit{0.431}  \best 0.186   
                2           \best 0.2   \best \textit{0.19}  \best 0.092   
wfg5 3obj 8dim  1         \best 1.274  \best \textit{0.699}  \best 0.428   
                2         \best 1.166  \best \textit{0.405}  \best 0.442   
wfg5 4obj 10dim 1         \best 2.624  \best \textit{0.839}  \best 1.557   
                2         \best 1.737   \best \textit{0.47}  \best 0.755   
wfg6 2obj 6dim  1         \best 0.439  \best \textit{0.172}  \best 0.191   
                2         \best 0.496   \best \textit{0.24}  \best 0.298   
wfg6 3obj 8dim  1         \best 1.172  \best \textit{0.723}  \best 0.403   
                2         \best 0.786   \best \textit{0.38}  \best 0.406   
wfg6 4obj 10dim 1         \best 2.333  \best \textit{0.928}  \best 0.661   
                2         \best 1.677   \best \textit{0.62}  \best 0.829   

                                                             \
                                                        100   
                                           IQR       median   
$f$             Position                                      
wfg1 2obj 3dim  1         \best \textit{0.199}        1.262   
                2                \textit{0.07}        0.981   
wfg1 3obj 4dim  1               \textit{0.253}        1.369   
                2         \best \textit{0.157}        1.444   
wfg1 4obj 5dim  1         \best \textit{0.165}  \best 1.776   
                2               \textit{0.129}  \best 1.831   
wfg2 2obj 6dim  1         \best \textit{0.305}  \best 0.148   
                2         \best \textit{0.508}  \best 0.893   
wfg2 3obj 6dim  1                \textit{0.87}  \best 1.338   
                2                \textit{0.36}  \best 0.833   
wfg2 4obj 10dim 1               \textit{0.632}         2.14   
                2         \best \textit{0.1

In [17]:
## summary table

In [18]:
def get_rank_table_from_list(LIST, attainable=False):
    D = {}
    for file_name in LIST:
        key = file_name[:-7]
        
        if attainable:
            score = get_rank_score_b(file_name)
        else:
            score = get_rank_score_a(file_name)
        if key not in D.keys(): 
            D[key] = score 
        else:
            D[key] += score 
            D[key] /= 2
    return D

def get_rank_score_a(file_name):
    TABLE_STAGES = [25, 50, 100, 150]
    result = load_result(file_name)
    rank_score = rank_comparison(result, intervals=TABLE_STAGES)
    return rank_score
    
    
def rank_comparison(resuts_container, intervals):
    M, M_ref, M_draw = get_comparison_matrices(resuts_container, intervals=intervals)
    score_1 = M*1
    score_2 = M_ref*0
    score_3 = M_draw*.5
    return (score_1+score_2+score_3).mean(axis=0)
    

def format_rank_df(df):
    df_new = df.round(3)
    cols = df_new.columns
    print(cols)
    new_cols = [c.split("_")[0].upper() for c in cols ]
    print(new_cols)
    df_new =df_new.rename(columns = {old: new for old, new in zip(cols, new_cols)}).round(3)
    
    # define win
    mask_win = (df_new.values>0.5)
    f_win = lambda value: "\\best "+value
    
    # define draw 
    mask_draw = (df_new.values==0.5)
    f_draw = lambda value: "\\statsimilar "+value
    
    # apply win/draw
    df_new = df_new.astype(str)
    apply_f_to_mask(df_new, mask_win, f_win)
    apply_f_to_mask(df_new, mask_draw, f_draw)

    return df_new

In [19]:
# overall ranked table
D = get_rank_table_from_list(RESULT_FILES_LIST, attainable=False)
df = pd.DataFrame.from_dict(D)
df_rank_overall = pd.DataFrame(df.values, columns=df.columns, index=["25", "50", "100", "150"])
df_rank_overall

,wfg1_2obj_3dim,wfg1_3obj_4dim,wfg1_4obj_5dim,wfg2_2obj_6dim,wfg2_3obj_6dim,wfg2_4obj_10dim,wfg3_2obj_6dim,wfg3_3obj_10dim,wfg3_4obj_10dim,wfg4_2obj_6dim,wfg4_3obj_8dim,wfg4_4obj_8dim,wfg5_2obj_6dim,wfg5_3obj_8dim,wfg5_4obj_10dim,wfg6_2obj_6dim,wfg6_3obj_8dim,wfg6_4obj_10dim
25,0.556452,0.473790,0.598790,0.580645,0.441532,0.590726,0.697581,0.596774,0.765121,0.609879,0.613911,0.677419,0.721774,0.792339,0.779234,0.843750,0.719758,0.731855
50,0.579637,0.463710,0.601815,0.663306,0.514113,0.712702,0.608871,0.664315,0.824597,0.613911,0.583669,0.667339,0.654234,0.912298,0.825605,0.743952,0.755040,0.940524
100,0.559476,0.414315,0.616935,0.700605,0.532258,0.563508,0.541331,0.807460,0.930444,0.580645,0.507056,0.704637,0.649194,0.978831,0.973790,0.630544,0.842742,0.985887
150,0.590726,0.392137,0.641129,0.761089,0.623992,0.500000,0.539819,0.719758,0.929435,0.587702,0.535282,0.723790,0.680444,0.970766,0.990927,0.566532,0.769153,0.939516


In [20]:
# 2obj
cols =[i for i in df.columns if strip_problem_names(i)[1]==2]
df_wfg_2obj = df[cols]
df_wfg_2obj = format_rank_df(df_wfg_2obj)
df_wfg_2obj

Index(['wfg1_2obj_3dim', 'wfg2_2obj_6dim', 'wfg3_2obj_6dim', 'wfg4_2obj_6dim',
       'wfg5_2obj_6dim', 'wfg6_2obj_6dim'],
      dtype='object')
['WFG1', 'WFG2', 'WFG3', 'WFG4', 'WFG5', 'WFG6']


,WFG1,WFG2,WFG3,WFG4,WFG5,WFG6
0,\best 0.556,\best 0.581,\best 0.698,\best 0.61,\best 0.722,\best 0.844
1,\best 0.58,\best 0.663,\best 0.609,\best 0.614,\best 0.654,\best 0.744
2,\best 0.559,\best 0.701,\best 0.541,\best 0.581,\best 0.649,\best 0.631
3,\best 0.591,\best 0.761,\best 0.54,\best 0.588,\best 0.68,\best 0.567


In [21]:
# 3 obj
cols =[i for i in df.columns if strip_problem_names(i)[1]==3]
df_wfg_3obj = df[cols]
df_wfg_3obj = format_rank_df(df_wfg_3obj)
df_wfg_3obj

Index(['wfg1_3obj_4dim', 'wfg2_3obj_6dim', 'wfg3_3obj_10dim', 'wfg4_3obj_8dim',
       'wfg5_3obj_8dim', 'wfg6_3obj_8dim'],
      dtype='object')
['WFG1', 'WFG2', 'WFG3', 'WFG4', 'WFG5', 'WFG6']


,WFG1,WFG2,WFG3,WFG4,WFG5,WFG6
0,0.474,0.442,\best 0.597,\best 0.614,\best 0.792,\best 0.72
1,0.464,\best 0.514,\best 0.664,\best 0.584,\best 0.912,\best 0.755
2,0.414,\best 0.532,\best 0.807,\best 0.507,\best 0.979,\best 0.843
3,0.392,\best 0.624,\best 0.72,\best 0.535,\best 0.971,\best 0.769


In [22]:
# 4 obj
cols =[i for i in df.columns if strip_problem_names(i)[1]==4]
df_wfg_4obj = df[cols]
df_wfg_4obj = format_rank_df(df_wfg_4obj)
df_wfg_4obj

Index(['wfg1_4obj_5dim', 'wfg2_4obj_10dim', 'wfg3_4obj_10dim',
       'wfg4_4obj_8dim', 'wfg5_4obj_10dim', 'wfg6_4obj_10dim'],
      dtype='object')
['WFG1', 'WFG2', 'WFG3', 'WFG4', 'WFG5', 'WFG6']


,WFG1,WFG2,WFG3,WFG4,WFG5,WFG6
0,\best 0.599,\best 0.591,\best 0.765,\best 0.677,\best 0.779,\best 0.732
1,\best 0.602,\best 0.713,\best 0.825,\best 0.667,\best 0.826,\best 0.941
2,\best 0.617,\best 0.564,\best 0.93,\best 0.705,\best 0.974,\best 0.986
3,\best 0.641,\statsimilar 0.5,\best 0.929,\best 0.724,\best 0.991,\best 0.94


In [23]:
# totals by objective
df_wfg_totals = pd.DataFrame(index=df_wfg_4obj.index)
cols_2obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==2]
df_wfg_totals[f"\n_dim = 2"] = df_rank_overall[cols_2obj].mean(axis=1).values
cols_3obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==3]
df_wfg_totals[f"\n_dim = 3"] = df_rank_overall[cols_3obj].mean(axis=1).values
cols_4obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==4]
df_wfg_totals[f"\n_dim = 4"] = df_rank_overall[cols_4obj].mean(axis=1).values
df_wfg_totals = df_wfg_totals.round(3)
df_wfg_totals

,\n_dim = 2,\n_dim = 3,\n_dim = 4
0,0.668,0.606,0.691
1,0.644,0.649,0.762
2,0.610,0.680,0.796
3,0.621,0.669,0.787


In [24]:
# totals by attainability 
df_wfg_totals = pd.DataFrame(index=df_wfg_4obj.index)
cols_2obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==2]
df_wfg_totals[f"\n_dim = 2"] = df_rank_overall[cols_2obj].mean(axis=1).values
cols_3obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==3]
df_wfg_totals[f"\n_dim = 3"] = df_rank_overall[cols_3obj].mean(axis=1).values
cols_4obj = [i for i in df_rank_overall.columns if strip_problem_names(i)[1]==4]
df_wfg_totals[f"\n_dim = 4"] = df_rank_overall[cols_4obj].mean(axis=1).values
df_wfg_totals

,\n_dim = 2,\n_dim = 3,\n_dim = 4
0,0.668347,0.606351,0.690524
1,0.643985,0.648858,0.762097
2,0.610299,0.680444,0.795867
3,0.621052,0.668515,0.787466


In [25]:
df_wfg_23obj = pd.concat([df_wfg_2obj, df_wfg_3obj], axis=1)
df_wfg_23obj.index = ["25", "50", "100", "150"] 
# save_table(df_wfg_23obj, "summary_table_03_2obj3obj_igd")
df_wfg_23obj

,WFG1,WFG2,WFG3,WFG4,WFG5,WFG6,WFG1,WFG2,WFG3,WFG4,WFG5,WFG6
25,\best 0.556,\best 0.581,\best 0.698,\best 0.61,\best 0.722,\best 0.844,0.474,0.442,\best 0.597,\best 0.614,\best 0.792,\best 0.72
50,\best 0.58,\best 0.663,\best 0.609,\best 0.614,\best 0.654,\best 0.744,0.464,\best 0.514,\best 0.664,\best 0.584,\best 0.912,\best 0.755
100,\best 0.559,\best 0.701,\best 0.541,\best 0.581,\best 0.649,\best 0.631,0.414,\best 0.532,\best 0.807,\best 0.507,\best 0.979,\best 0.843
150,\best 0.591,\best 0.761,\best 0.54,\best 0.588,\best 0.68,\best 0.567,0.392,\best 0.624,\best 0.72,\best 0.535,\best 0.971,\best 0.769


In [26]:
df_wfg_4Tobj = pd.concat([df_wfg_4obj, df_wfg_totals], axis=1)
df_wfg_4Tobj.index = ["25", "50", "100", "150"] 
cols = pd.MultiIndex.from_tuples(zip((["\n_dim =4"]*6)+(["Totals"]*3), df_wfg_4Tobj.columns))
df_wfg_4Tobj.columns = cols

# save_table(df_wfg_4Tobj, "summary_table_04_4objTotal_igd")
df_wfg_4Tobj

\n_dim =4                                                           \
            WFG1              WFG2         WFG3         WFG4         WFG5   
25   \best 0.599       \best 0.591  \best 0.765  \best 0.677  \best 0.779   
50   \best 0.602       \best 0.713  \best 0.825  \best 0.667  \best 0.826   
100  \best 0.617       \best 0.564   \best 0.93  \best 0.705  \best 0.974   
150  \best 0.641  \statsimilar 0.5  \best 0.929  \best 0.724  \best 0.991   

                     Totals                        
            WFG6 \n_dim = 2 \n_dim = 3 \n_dim = 4  
25   \best 0.732   0.668347   0.606351   0.690524  
50   \best 0.941   0.643985   0.648858   0.762097  
100  \best 0.986   0.610299   0.680444   0.795867  
150   \best 0.94   0.621052   0.668515   0.787466